In [3]:
import requests
import dill
from bs4 import BeautifulSoup
from datetime import datetime
import time
import spacy

In [4]:
headers = { 'user-agent' : 'TDI' }

## Get links for all pages

In [ ]:
url='https://web.archive.org/web/20150913224145/http://www.newyorksocialdiary.com/party-pictures'
LIMIT = 25
def get_page_args(i):
    return {"url": url,
            "params": {"page": i}}
def get_page(i):
    return requests.get(**get_page_args(i),headers=headers)
responses=[get_page(i) for i in range(1,LIMIT+1)]

In [ ]:
def get_links(response):
    soup = BeautifulSoup(response.text, "lxml")
    content=soup.find('div',attrs={'class':'view-content'})
    ##links
    links=content.find_all('a')
    urls=[url['href'] for url in links]
    new_links=['https://web.archive.org/'+l for l in urls]
    ##time
    time=content.find_all('span',attrs={'class':'views-field views-field-created'})
    times=[t.text for t in time]
    dates= [datetime.strptime(time, " %A, %B %d, %Y ") for time in times]
    
    lis=list(zip(new_links,dates))
    return lis # A list of URL, date pairs

link_list=[get_links(response) for response in responses]


In [ ]:
##store the lists
link_list = dill.load(open('nysd-links.pkd', 'rb'))

In [ ]:
#get rid of time
links=[link[0] for link in link_list]

### get captions for a page and a list of pages

In [ ]:
##get all captions for a page
def get_captions(path):
    page = requests.get(path,headers=headers)
    soup = BeautifulSoup(page.text, "lxml")
    caption_divs_main = soup.find_all('font', attrs={'face': 'Verdana, Arial, Helvetica, sans-serif'},text=True) 
    caption_divs2 = soup.find_all("div", {'align': 'center', 'class': 'photocaption'}, text=True)
    caption_divs3 = soup.find_all("td", {'valign': 'top', 'class': 'photocaption'}, text=True)
    caption_divs4 = soup.find_all("td", {'valign': 'top', 'class': 'photocaption', 'style': 'background-color:#faf9ee'}, text=True)
    caption_divs = caption_divs_main + caption_divs2 + caption_divs3 + caption_divs4
    lis=[c.text.strip() for c in caption_divs]
    return lis

In [ ]:
##get captions for each picture for each page
def get_pic_names(link):
    captions=[]
    try:
        caps=get_captions(link)
    except:
        time.sleep(5)
        try:
            caps=get_captions(link)
        except:
            pass
    captions.extend(caps)
    return captions

In [ ]:
##get names for all links
names=[]
ind=[]
for i,link in enumerate(links):
    try:
        x=get_pic_names(link)
    except:
        time.sleep(2)
        try:
            x=get_pic_names(link)
        except: 
            continue
    names.extend(x)

In [ ]:
##try to use spacy but not working
import spacy
nlp = spacy.load("en_core_web_sm")
captions=[]
for n in names:
    name=[]
    doc=nlp(n)
    for ent in doc.ents:
        name.append((ent.text, ent.label_))        
    captions.append([n[0] for n in name if n[1]=="PERSON"])

In [ ]:
len(names)

In [ ]:
dill.dump(names,open('names_sunday.pkd', 'wb'))

In [7]:
captions = dill.load(open('names_sunday.pkd', 'rb'))

In [ ]:
len(captions)
captions[:100]

### write functions to extract names

In [ ]:
nlp = spacy.load("en_core_web_sm")
replace_list=['The','Dr','Verdana','Helvetica','Chairman','Director','Guests','President','Co-Chair','Hospital',
              'Special Surgery','Executive Director','New York','Museum','Photographs','Children','The Society',
              'Dinner','Board Member','Castle','Skip','Michael R. Bloomberg']

In [5]:
def extract_names(captions):
    names_extract=[]
    for caption in captions:
        #print(caption)
        ##get rid of extra space
        if len(str(caption))>250:
            return []
        else:
            name=' '.join(caption.split())
            
            ### replace unnecessary titles
            for word in replace_list:
                if word in name:
                    name=name.replace(word,'')
        ##Deal with couple names
         ##get the couple names
            if (len(name.split()) == 4) and (name.split()[1] == 'and'):
                couple=name.split()
                couple.insert(1,couple[3])
                name=' '.join(couple)
            
                ###get rid of couple names in a whole sentence
            if ',' in name:
                names_sub=str(name).split(',')
                new_list=[]
                for n in names_sub:
                    n_parts = n.split()
                    if n_parts and (n_parts[0]=='and'):
                        new_n=n_parts
                        del new_n[0]
                        n_temp=' '.join(new_n)
                        if (len(n_temp.split()) == 4) and (n_temp.split()[1] == 'and'):
                            couple=n_temp.split()
                            couple.insert(1,couple[3])
                            n_temp=' '.join(couple)
                        new_list.append(n_temp)
                    else:
                        if (len(n.split()) == 4) and (n.split()[1] == 'and'):
                            couple=n.split()
                            couple.insert(1,couple[3])
                            n=' '.join(couple)        
                        new_list.append(n)
                    name=','.join(new_list)

           ##use spacy for the rest of the work
            name_temp=[]
            doc=nlp(name)
            for ent in doc.ents:
                name_temp.append((ent.text, ent.label_))        
            pure_names=[n[0] for n in name_temp if n[1]=="PERSON"]
            pure_names_strip=[' '.join(n.split()) for n in pure_names]
            names_extract.append(pure_names_strip)
    return names_extract

In [ ]:
##comment:
##get rid of for
##if len() >250, return []
##put extra space
##write separate functions
###words_to_remove=[a list of words]
#for word in words_to_remove:
 #   if word in name:
  #      name=name.replace(word,'')
##use n_parts = n.split() to replace n.split()
### if n_parts: everything is true except empty,Nonetype, dont have to check len(n_parts)
##shift+tab unindent, tab: indent

In [8]:
##extract all the names
name_revised1=extract_names(captions)
dill.dump(name_revised1,open('name_revised1.pkd', 'wb'))

In [10]:
name_revised1

[[],
 ['Les Lieberman',
  'Barri Lieberman',
  'Isabel Kallman',
  'Trish Iervolino',
  'Ron Iervolino'],
 ['Chuck Grodin'],
 ['Diana Rosario',
  'Ali Sussman',
  'Sarah Boll',
  'Jen Zaleski',
  'Alysse Brennan',
  'Lindsay Macbeth'],
 ['Kelly Murro', 'Tom Murro'],
 [],
 ['Ron Iervolino', 'Trish Iervolino', 'Russ Middleton', 'Lisa Middleton'],
 ['Barbara Loughlin', 'Gerald Loughlin', 'Debbie Gelston'],
 ['Julianne Michelle'],
 ['Heather Robinson',
  'Kiwan Nichols',
  'Jimmy Nichols',
  'Melanie Carbone',
  'Nancy Brown'],
 ['Bill Mack', 'Les Lieberman'],
 ['David Lyden', 'Patricia Sorenson'],
 ['Jimmy Cayne', 'Pat Cayne'],
 ['Stuart Oran', 'Les Lieberman', 'Hilary Oran'],
 ['Chuck Grodin'],
 ['Dwight Gooden', 'Les Lieberman'],
 ['Amy Cunningham-Bussel', 'Ray Mirra', 'Tyler Janovitz'],
 ['Dan Shedrick', 'Samara Heafitz'],
 ['Cass Adelman', 'Jason Adelman'],
 ['Bart Scott', 'Mark Laplander'],
 ['Mitch Rubin',
  'Audra Zuckerman',
  'Michelle Smith',
  'Kenneth Mehlman',
  'Julia Harqua

In [ ]:
###check randonly whether the extract function works
import random
r=random.randint(0,len(captions))
print(captions[r:r+10])
extract_names(captions[r:r+10])


In [263]:
##trying to figure out the couple names, and when its in the sentence
lis=['Katie Tozer and boys']
lis1=['Ali and Marina Wambold, Jeanne Lawrence, and Donald and Barbara Tober']

In [218]:
names_sub=str(lis1).split(',')
new_list=[]
for n in names_sub:
    n=n.strip()
    if n.split()[0]=='and':
        new_n=n.split()
        del new_n[0]
        n_temp=' '.join(new_n)
        if (len(n_temp.split()) == 4) and (n_temp.split()[1] == 'and'):
            couple=n_temp.split()
            couple.insert(1,couple[3])
            n_temp=' '.join(couple)
        new_list.append(n_temp)
    else:
        if (len(n.split()) == 4) and (n.split()[1] == 'and'):
            couple=n.split()
            couple.insert(1,couple[3])
            n=' '.join(couple)        
        new_list.append(n)
new_list

["['Ali Wambold and Marina Wambold",
 'Jeanne Lawrence',
 "Donald Tober'] and Barbara Tober']"]

In [212]:
if ',' in lis:
    print(lis)

In [244]:
extract_names(lis1)

[['Ali Wambold',
  'Marina Wambold',
  'Jeanne Lawrence',
  'Donald Tober',
  'Barbara Tober']]

### Find the most popular ones

In [252]:
import itertools  # itertools.combinations may be useful
import networkx as nx
G=nx.MultiGraph()
from itertools import combinations
lis=[]
for n in name_revised1:
    a = combinations(n, 2)
    lis.extend(a)
len(lis)

181963

In [253]:
G.add_edges_from(lis)

In [254]:
q3=sorted(list(G.degree()),key=lambda x: (-x[1]))[:100]

q3

[('Jean Shafiroff', 614),
 ('Gillian Miniter', 495),
 ('Mark Gilbertson', 447),
 ('Geoffrey Bradfield', 338),
 ('Alexandra Lebenthal', 307),
 ('Andrew Saffir', 280),
 ('Debbie Bancroft', 277),
 ('Eleanora Kennedy', 274),
 ('Yaz Hernandez', 269),
 ('Sharon Bush', 264),
 ('Kamie Lightburn', 247),
 ('Jamee Gregory', 240),
 ('Michael Bloomberg', 236),
 ('Alina Cho', 234),
 ('Muffie Potter Aston', 231),
 ('Mario Buatta', 228),
 ('Bonnie Comley', 218),
 ('Lucia Hwong Gordon', 217),
 ('Karen LeFrak', 215),
 ('Barbara Tober', 211),
 ('Allison Aston', 210),
 ('Bettina Zilkha', 201),
 ('John', 191),
 ('Deborah Norville', 188),
 ('Patrick McMullan', 183),
 ('Liz Peek', 181),
 ('Evelyn Lauder', 177),
 ('Fernanda Kellogg', 175),
 ('Daniel Benedict', 175),
 ('Stewart Lane', 175),
 ('Michele Herbert', 171),
 ('Martha Stewart', 167),
 ('Audrey Gruss', 165),
 ('Elizabeth Stribling', 164),
 ('Dennis Basso', 161),
 ('Nicole Miller', 161),
 ('Sylvester Miniter', 160),
 ('Barbara Regna', 160),
 ('Amy Fine 

In [258]:
import numpy as np
np.max([lis[1] for lis in q3])

614

## Find the pagerank

In [ ]:
###pagerank, matters with weight!!!!!
Gg = nx.Graph()

for u,v,data in G.edges(data=True):
    if Gg.has_edge(u,v):
        Gg[u][v]['weight'] += 1
    else:
        Gg.add_edge(u, v, weight=1)
    # get pagerank, 0.85 is default
pr=nx.pagerank(Gg)
page_rank=[]
for k,v in pr.items():
    x=(k,v)
    page_rank.append(x)

q4=sorted(page_rank,key=lambda x: -x[1])[:100]

## Find the best friends

In [ ]:
###best friends
H=nx.Graph()
H.add_weighted_edges_from(G.edges)
friends=sorted(H.edges(data=True),key= lambda x: x[2]['weight'],reverse=True)[:100]
q5=[]
for friend in friends:
    x=((friend[0],friend[1]),friend[2]['weight'])
    q5.append(x)
q5